In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

class LDA:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.class_means = [np.mean(X[y == c], axis=0) for c in self.classes]
        self.priors = [np.mean(y == c) for c in self.classes]
        self.shared_covariance = np.cov(X.T)

    def predict(self, X):
        discriminants = []
        for i, c in enumerate(self.classes):
            mean_diff = X - self.class_means[i]
            cov_inv = np.linalg.inv(self.shared_covariance)
            w = np.dot(cov_inv, (self.class_means[i] - np.mean(self.class_means, axis=0)))
            b = -0.5 * np.dot(np.dot(self.class_means[i].T, cov_inv), self.class_means[i]) + np.log(self.priors[i])
            discriminant = np.dot(mean_diff, w) + b
            discriminants.append(discriminant)
        return self.classes[np.argmax(discriminants, axis=0)]


class QDA:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.means = [np.mean(X[y == c], axis=0) for c in self.classes]
        self.covariance_matrices = [np.cov(X[y == c].T) for c in self.classes]
        self.priors = [np.mean(y == c) for c in self.classes]

    def predict(self, X):
        discriminants = []
        for i, c in enumerate(self.classes):
            mean_diff = X - self.means[i]
            cov_inv = np.linalg.inv(self.covariance_matrices[i])
            term1 = -0.5 * np.sum(np.dot(mean_diff, cov_inv) * mean_diff, axis=1)
            term2 = -0.5 * np.log(np.linalg.det(self.covariance_matrices[i]))
            term3 = np.log(self.priors[i])
            discriminant = term1 + term2 + term3
            discriminants.append(discriminant)
        return self.classes[np.argmax(discriminants, axis=0)]

# Carregar conjunto de dados Iris
iris = load_iris()
X, y = iris.data, iris.target

# Dividir os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Instanciando e treinado os classificadores LDA
lda = LDA()
lda.fit(X_train_scaled, y_train)
# Instanciando e treinado os classificadores LDA
qda = QDA()
qda.fit(X_train_scaled, y_train)

# Fazer a predição para o LDA
y_train_pred_lda = lda.predict(X_train_scaled)
y_test_pred_lda = lda.predict(X_test_scaled)
# Fazer a predição para o QDA
y_train_pred_qda = qda.predict(X_train_scaled)
y_test_pred_qda = qda.predict(X_test_scaled)

# Calcular a acurácia LDA
accuracy_train_lda = accuracy_score(y_train, y_train_pred_lda)
accuracy_test_lda = accuracy_score(y_test, y_test_pred_lda)
# Calcular a acurácia QDA
accuracy_train_qda = accuracy_score(y_train, y_train_pred_qda)
accuracy_test_qda = accuracy_score(y_test, y_test_pred_qda)

print("Acurácia no conjunto de treino (LDA): {:.2f}%".format(accuracy_train_lda * 100))
print("Acurácia no conjunto de teste (LDA): {:.2f}%".format(accuracy_test_lda * 100))
print("Acurácia no conjunto de treino (QDA): {:.2f}%".format(accuracy_train_qda * 100))
print("Acurácia no conjunto de teste (QDA): {:.2f}%".format(accuracy_test_qda * 100))


Acurácia no conjunto de treino (LDA): 69.17%
Acurácia no conjunto de teste (LDA): 70.00%
Acurácia no conjunto de treino (QDA): 97.50%
Acurácia no conjunto de teste (QDA): 96.67%
